# Домашнее задание №1

In [155]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from nltk.corpus import stopwords
import RAKE
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

## 1. Сбор данных

In [4]:
ua = UserAgent(verify_ssl=False)

In [75]:
articles = ['https://ria.ru/20211109/egipet-1758078826.html',
            'https://ria.ru/20211031/zabroshki-1756883477.html',
            'https://ria.ru/20211029/aviabilety-1756545652.html',
            'https://ria.ru/20211023/venesuela-1755854229.html']

In [112]:
all_articles = []
for art in tqdm(articles):
    art_dict = {}
    req = requests.get(art, headers={'User-Agent': ua.random})
    soup = BeautifulSoup(req.text, 'html.parser')
    inf = soup.find_all('div', {'class': 'article__text'})
    art_dict['text'] = ' '.join([i.text for i in inf])
    keywords_inf = soup.find('meta', {'name': 'keywords'})
    art_dict['keywords'] = keywords_inf['content'].split(', ')
    all_articles.append(art_dict)

100%|█████████████████████████████████████████████| 4/4 [00:04<00:00,  1.16s/it]


## 2. Ручная разметка и конкатенация двух наборов ключевых слов

In [113]:
my_keywords = [['безопасность', 'египет', 'пять звезд', 'отдых', 'рейс', 'авиакомпания', 
                'коронавирусная инфекция', 'гостиница', 'аэропорт', 'отель', 'чартер'],
               ['заброшка', 'здание', 'атмосфера', 'экскурсия', 'россия', 'покинутое здание'],
               ['экономия', 'эффект вторника', 'билет', 'турист', 'рейс', 'стыковочный рейс', 
                'лоукостер', 'авиабилет', 'коронавирус', 'маршрут', 'хорошая цена'],
               ['тропический остров маргарита', 'остров', 'тропический остров', 'маргарита', 
                'венесуэла', 'курорт', 'деньги', 'интернет', 'отдых', 'песчаный пляж', 'пляж', 
                'достопримечательность', 'музей', 'местная кухня', 'кухня', 'беспошлинная торговля']]

In [114]:
def concat_keywords(art_dict, keys):
    print('текст:', art_dict['text'][:200] + '...')
    print('мои ключевые слова:', keys)
    print('ключевые слова с сайта:', art_dict['keywords'])
    art_dict['keywords'] = list(set(art_dict['keywords'] + keys))
    print('все ключевые слова (золотой стандарт):', art_dict['keywords'], '\n')
    return art_dict

In [115]:
for i in range(len(all_articles)):
    concat_keywords(all_articles[i], my_keywords[i])

текст: МОСКВА, 9 ноя — РИА Новости, Светлана Баева. Сегодня, спустя шесть лет после крушения авиалайнера, следовавшего из Шарм-эш-Шейха в Санкт-Петербург, на курорты Египта из России снова начинают летать ча...
мои ключевые слова: ['безопасность', 'египет', 'пять звезд', 'отдых', 'рейс', 'авиакомпания', 'коронавирусная инфекция', 'гостиница', 'аэропорт', 'отель', 'чартер']
ключевые слова с сайта: ['египет', 'шарм-эш-шейх', 'хургада', 'маршруты - туризм', 'самолеты', 'авиакомпании', 'туристы', 'отели', 'туризм', 'куда поехать', 'отдых на море', 'куда можно лететь']
все ключевые слова (золотой стандарт): ['египет', 'туризм', 'куда можно лететь', 'авиакомпания', 'туристы', 'чартер', 'авиакомпании', 'шарм-эш-шейх', 'отдых на море', 'пять звезд', 'отдых', 'безопасность', 'самолеты', 'куда поехать', 'рейс', 'маршруты - туризм', 'гостиница', 'хургада', 'коронавирусная инфекция', 'отель', 'отели', 'аэропорт'] 

текст: МОСКВА, 31 окт — РИА Новости, Светлана Баева. Выбитые окна зияют темными гла

### Препроцессинг

In [116]:
stops = stopwords.words('russian')

In [117]:
m = MorphAnalyzer()
def preprocess(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

In [118]:
for art in all_articles:
    art['text'] = preprocess(art['text'])
    key_lemmas = []
    for k in art['keywords']:
        key_lemmas.append(preprocess(k))
    art['keywords'] = list(set(key_lemmas))

## 3. Автоматическое извлечение ключевых слов

### RAKE

In [132]:
rake = RAKE.Rake(stops)

In [137]:
for art in all_articles:
    RAKE_keywords = rake.run(art['text'], maxWords=3, minFrequency=2)
    art['RAKE_keywords'] = [r[0] for r in RAKE_keywords if r[1] >= 1]

### TextRank

In [150]:
for art in all_articles:
    TextRank_keywords = keywords.keywords(art['text'], 
                                          language='russian', 
                                          additional_stopwords=stops, 
                                          scores=True)
    art['TextRank_keywords'] = [r[0] for r in TextRank_keywords]

### TF-IDF

In [219]:
texts = [a['text'] for a in all_articles]
vectorizer = TfidfVectorizer(stop_words=stops, ngram_range=(1,3))
X = vectorizer.fit_transform(texts)

In [220]:
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [251]:
for i in range(len(all_articles)):
    all_articles[i]['TfIdf_keywords'] = df.loc[i][df.loc[i] > 0.07].index.tolist()

## 4. Морфологический шаблон

Мы обсуждали, что ключевые слова — это часто имена существительные. В моём золотом стандарте мало биграмм и слов других частей речи, поэтому мой морфологический шаблон — **NOUN** (только униграммы). Зато это будет хотя бы насколько-то что-то показывать.

In [277]:
for art in tqdm(all_articles):
    new_dict = {}
    for key in art:
        if key.endswith('keywords'):
            templated = []
            for keyword in art[key]:
                if m.parse(keyword)[0].tag.POS == 'NOUN':
                    templated.append(keyword)
            new_dict[key + '_NOUN'] = templated
    art.update(new_dict)

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 29.65it/s]


## 5. Подсчёт метрик

In [322]:
def metrics(art, method, golden):
    predicted = art[method]
    TP = len(list(set(golden) & set(predicted)))
    precision = TP / len(predicted)
    metrics_dict['precision'] += precision
    recall = TP / len(golden)
    metrics_dict['recall'] += recall
    metrics_dict['F1'] += 2 * (precision * recall) / (precision + recall)

In [323]:
for method in list(all_articles[0]):
    if method in ['text', 'keywords', 'keywords_NOUN']:
        continue
    metrics_dict = {'precision': 0, 'recall': 0, 'F1': 0}
    for art in all_articles:
        if method.endswith('_keywords'):
            golden = art['keywords']
            metrics(art, method, golden)
        elif method.endswith('_keywords_NOUN'):
            golden = art['keywords_NOUN']
            metrics(art, method, golden)
    print('метод:', method)
    print('точность:', round(metrics_dict['precision'] / len(all_articles), 2))
    print('полнота:', round(metrics_dict['recall'] / len(all_articles), 2))
    print('F1:', round(metrics_dict['F1'] / len(all_articles), 2))
    print('\n')

метод: RAKE_keywords
точность: 0.16
полнота: 0.11
F1: 0.12


метод: TextRank_keywords
точность: 0.07
полнота: 0.23
F1: 0.11


метод: TfIdf_keywords
точность: 0.25
полнота: 0.14
F1: 0.17


метод: RAKE_keywords_NOUN
точность: 0.23
полнота: 0.12
F1: 0.15


метод: TextRank_keywords_NOUN
точность: 0.14
полнота: 0.27
F1: 0.19


метод: TfIdf_keywords_NOUN
точность: 0.3
полнота: 0.16
F1: 0.21




## 6. Ошибки и решения

In [327]:
print('золотой стандарт:', all_articles[0]['keywords'])
print('предсказал RAKE:', all_articles[0]['RAKE_keywords'])

золотой стандарт: ['египет', 'туризм', 'куда можно лететь', 'авиакомпания', 'пять звезда', 'чартер', 'турист', 'шарм-эш-шейх', 'коронавирусный инфекция', 'отдых', 'безопасность', 'куда поехать', 'рейс', 'гостиница', 'маршрут - туризм', 'отдых на мор', 'отель', 'хургад', 'самолёт', 'аэропорт']
предсказал RAKE: ['шарм-эш-шейх', 'весь включить', 'курорт египет', 'санкт-петербург', 'anex tour', 'tui россия', 'россия', 'число', 'хургад', 'москва', 'человек', 'звезда', 'рейс', 'безопасность', 'интурист', 'неделя', 'екатеринбург', 'казань', 'компания', 'солнце']


* К сожалению, кажется, что использованные методы выделения ключевых слов зачастую чувствительны к именам собственным, которые не всегда бывают важны для конкретного текста. Возможно, для больших текстов имеет смысл сначала осуществить суммаризацию, а затем выделить ключевые слова, чтобы не останавливаться на чём-то на самом деле не слишком важном.
* Кроме того, есть необходимость в расширении использованного списка стоп-слов, даже нейтральные слова типа *хороший* и *человек* могут ухудшать качество.
* Контекстуализованные модели (например, BERT) также могут стать неплохим помощником в выделении ключевых слов. Кажется, что контекст действительно важен. Насколько я знаю, таковые уже существуют.